In [1]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime

In [2]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

In [3]:
# Recuperar empresas ID de salesforce
results = sf.query_all(
    """
   select Id, Name, IdentificationId__pc, Identification_Type__c,Holding__c ,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,poliza__c,Empleador__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    ["Id", "Name", "IdentificationId__pc", "Identification_Type__c", "Holding__c", "Pa_s_del_Paciente__c",'Campa_a__c','AccountNumber','Id_Empresa__c','Empleador__c','poliza__c']
]
Empresas_SF.shape

(7875, 11)

In [4]:
# recuperar todos los pacientes ID
results = sf.query_all(
    """
   select Id, IdentificationId__pc,Name from Account 
    """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[["Id", "IdentificationId__pc", "Name"]]
Pacientes_SF.shape

(1540197, 3)

In [10]:
results = sf.query_all(
    """
   select Id,AccountId,ContactId,NameAccount__c, RutContact__c from AccountContactRelation
    """
)
Relacion_Pacientes_Empresas = pd.DataFrame(results["records"])

Relacion_Pacientes_Empresas = Relacion_Pacientes_Empresas[
    ["Id", "AccountId", "ContactId", "NameAccount__c", "RutContact__c"]
]
Relacion_Pacientes_Empresas.shape

(1220291, 5)

In [15]:
Union_relacion_paciente = pd.merge(
    left=Relacion_Pacientes_Empresas,
    right=Pacientes_SF,
    how="left",
    left_on="RutContact__c",
    right_on="IdentificationId__pc",
)
Union_relacion_paciente.head()

,Id_x,AccountId,ContactId,NameAccount__c,RutContact__c,Id_y,IdentificationId__pc,Name
0,07k8c00000RGWPAAA5,0018c00002axQdlAAE,0038c000036jelbAAA,Metlife Holding Metlife Skberge,182204846,0018c00002cVWElAAO,182204846,Maldonado Velasquez Nicolas
1,07k8c00000RGWPBAA5,0018c00002axQdlAAE,0038c000035CWLYAA4,Metlife Holding Metlife Skberge,18396206K,0018c00002amMfoAAE,18396206K,Zaror Yunis Daniel Esteban
2,07k8c00000RGWPCAA5,0018c00002axQdlAAE,0038c000035CaxiAAC,Metlife Holding Metlife Skberge,184262452,0018c00002amRHeAAM,184262452,Bustamante Morales Flor Del Carmen
3,07k8c00000RGWPDAA5,0018c00002axQdlAAE,0038c000036jelcAAA,Metlife Holding Metlife Skberge,184429772,0018c00002cVWEmAAO,184429772,Malleo Abarca Rigoberto Andres
4,07k8c00000RGWPKAA5,0018c00002axQa5AAE,0038c000036jLDoAAM,Metlife Holding Metlife Finning,233986291,0018c00002cVCmMAAW,233986291,Albanez Gonzalez Matias Oliver Franci


In [14]:
Union_relacion_paciente.shape

(1220291, 8)

In [13]:
Union_relacion_paciente_empresa = pd.merge(
    left=Union_relacion_paciente,
    right=Empresas_SF,
    how="left",
    left_on="AccountId",
    right_on="Id",
)

Union_relacion_paciente_empresa.head()

,Id_x,AccountId,ContactId,NameAccount__c,RutContact__c,Id_y,IdentificationId__pc_x,Name_x,Id,Name_y,IdentificationId__pc_y,Identification_Type__c,Holding__c,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,Empleador__c,poliza__c
0,07k8c00000RGWPAAA5,0018c00002axQdlAAE,0038c000036jelbAAA,Metlife Holding Metlife Skberge,182204846,NaN,NaN,NaN,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,Holding Metlife,a1U8c000005dAtnEAE,Metlife,None,EMP5641,None,None
1,07k8c00000RGWPBAA5,0018c00002axQdlAAE,0038c000035CWLYAA4,Metlife Holding Metlife Skberge,18396206K,NaN,NaN,NaN,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,Holding Metlife,a1U8c000005dAtnEAE,Metlife,None,EMP5641,None,None
2,07k8c00000RGWPCAA5,0018c00002axQdlAAE,0038c000035CaxiAAC,Metlife Holding Metlife Skberge,184262452,NaN,NaN,NaN,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,Holding Metlife,a1U8c000005dAtnEAE,Metlife,None,EMP5641,None,None
3,07k8c00000RGWPDAA5,0018c00002axQdlAAE,0038c000036jelcAAA,Metlife Holding Metlife Skberge,184429772,NaN,NaN,NaN,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,Holding Metlife,a1U8c000005dAtnEAE,Metlife,None,EMP5641,None,None
4,07k8c00000RGWPKAA5,0018c00002axQa5AAE,0038c000036jLDoAAM,Metlife Holding Metlife Finning,233986291,NaN,NaN,NaN,0018c00002axQa5AAE,Metlife Holding Metlife Finning,None,None,Holding Metlife,a1U8c000005dAtnEAE,Metlife,None,EMP5951,None,None
